# Pregunta 22

Top 5 de lenguajes que son usados por usuarios bilingües.

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

21/10/30 14:00:52 WARN Utils: Your hostname, Ideapad3 resolves to a loopback address: 127.0.1.1; using 192.168.0.237 instead (on interface wlp0s20f3)
21/10/30 14:00:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/30 14:00:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/luis/anaconda3/envs/organizacionDatos/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
languages = sqlContext.read.csv('./Wikipedia_dataset_2c2021/languages.csv', 
                                    header=True, inferSchema=True, multiLine=True)
languages_rdd = languages.rdd

## Filtro los usuarios bilingues

Un usuario es Bilingue si conoce 2 idiomas.

Cuento la cantidad de idiomas que usa cada usuario y luego filtro unicamente aquellos usuarios que son bilingues.

In [4]:
usuarios_bilingues = languages_rdd.map(lambda x: (x.babel_user,1)).reduceByKey(lambda x,y: x+y)\
            .filter(lambda x: x[1]==2)

In [5]:
usuarios_bilingues.take(5)

[(1801431, 2), (1724336, 2), (1725400, 2), (674716, 2), (4334792, 2)]

## Analizo los idiomas que usan los usuarios bilingues

Del RDD de languages filtro los usuarios bilingues y determino los idiomas mas populares entre estos usuarios.

In [6]:
languages_rdd.join(usuarios_bilingues).map(lambda x: (x[1][0],1)).reduceByKey(lambda x,y: x+y).\
            takeOrdered(5, lambda x: -x[1])

[('es', 3054), ('en', 2944), ('fr', 56), ('ca', 54), ('pt', 22)]

# Conclusion

Observamos que los 5 idiomas mas hablados por los usuarios bilingues de Wikipedia son:

| Idioma | Cantidad de Usuarios | 
|--------------|-----------|
| Es |  3054 |
| En | 2944  |
| Fr | 56  |
| Ca | 54  |
| Pt | 22  |

Vemos que una mayoria de los usuarios bilingues dominan el ingles y español. 

Esto hace sentido pues gran parte de la comunidad hispano hablante termina dominando el ingles por razones laburales, academicas u otro motivo, antes que cualquier otro idioma.